# Pràctica 4: Recomanacions amb la llibreria Surprise

David Galera

#Heu de fer un Notebook que recomani pel·lícules fent servir la llibreria Surprise de Python. Ha de permetre introduir el codi d’un usuari i ha de donar com a sortida una llista de les N pel·lícules que més han agradat a l’usuari juntament amb tres llistes (fent servir user-user CF, item-item CF i SVD) de N recomanacions de pel·lícules que l’usuari no hagi ja puntuat, ordenades per ordre de puntuació estimada. No es poden repetir pel·lícules entre les dues llistes. Compareu els errors i els resultats fent servir cadascun dels tres mètodes.

In [1]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3193678 sha256=4a3afd5ef6e33f463fb12b6e4f4602ad610648b2e90e04dd84f0b71947fc6810
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import surprise


In [4]:
ratings_data = pd.read_csv('/content/drive/MyDrive/Advanced ML/ratings.csv')[['userId', 'movieId', 'rating']]
movies_metadata = pd.read_csv('/content/drive/MyDrive/Advanced ML/movies.csv')
print(ratings_data.shape)
ratings_data.head()


(100836, 3)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [5]:
movies_metadata.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Creem un DataFrame amb els camps **userId, movieId i rating**. Fent servir el títol de la pel·lícula com a index:

In [6]:
merged = pd.merge(ratings_data,movies_metadata[['movieId','title']],on=['movieId'],how='left')
merged.set_index('title',inplace=True)
merged.head()

,userId,movieId,rating
title,,,
Toy Story (1995),1,1,4.0
Grumpier Old Men (1995),1,3,4.0
Heat (1995),1,6,4.0
Seven (a.k.a. Se7en) (1995),1,47,5.0
"Usual Suspects, The (1995)",1,50,5.0


Definim una funció que realitza el següent: 
1. Demana introduïr l'**id** d'un usuari i el número **N** de recomanacions que és volen.
2. Crea una llista amb les pel·lícules no vistes per l'usuari.
3. Imprimeix les **N** pel·lícules que l'usuari ha puntuat més alt.
4. Instància un objecte KNN amb veïnatge 30 i **métode de filtratge col·laboratiu user-user**, l'entrena amb tot el dataset i fa prediccions de totes les pel·lícules no puntuades per l'usuari.
5. Imprimeix les **N** prediccions més altes.
6. Repeteix els passos 4 i 5 amb un objecte de veïnatge 20 i **métode de filtratge col·laboratiu item-item**.
7. Repeteix els passos 4 i 5 amb un objecte **SVD**. 

In [9]:
def recommendations():
  from operator import itemgetter
  from surprise import KNNBasic
  from surprise import SVD
  from surprise import Dataset
  from surprise import Reader
  print('Escriu userId: ')
  Id=int(input())
  print('Número de títols recomanats: ')
  N=int(input())
  #Id=1
  #N=10
  all_filmsId=list(movies_metadata.movieId.unique())
  watched_filmsId=ratings_data[ratings_data['userId']==Id]['movieId']#Pel·lícules vistes per l'usuari
  NOwatched_filmsId=list(set(all_filmsId) - set(watched_filmsId))#Pel·lícules no vistes per l'usuari
  print("Pel·lícules que més han agradat a l'usuari")
  print('--------------------------------------------------------')
  for i in merged[merged['userId']==Id]['rating'].nlargest(N).index:
    print(i)
  print('--------------------------------------------------------')
  reader = Reader(rating_scale=(1, 5))
  data = Dataset.load_from_df(ratings_data, reader)
  # Retrieve the trainset.
  trainset = data.build_full_trainset()
  # KNN user-user
  uu = KNNBasic(verbose=False,k=30,min_k=30, sim_options={'name': 'pearson', 'user_based': True, 'verbose' : True})#Veinatge de 30
  uu.fit(trainset)
  prediction={}
  for novista in NOwatched_filmsId:
    title=movies_metadata[movies_metadata.movieId==novista]['title'].values[0]
    prediction[title]=uu.predict(uid=Id, iid=novista).est
  print("Llistat de títols recomanats amb el métode user-user")
  print('--------------------------------------------------------')
  if N>len(prediction):#màxim titols recomanats és el total de pel·lícules predites
    N=len(prediction)
  for recomanacio in dict(sorted(prediction.items(), key = itemgetter(1), reverse = True)[:N]):
    print(recomanacio,', Predicció rating: ',dict(sorted(prediction.items(), key = itemgetter(1), reverse = True))[recomanacio])
  # KNN item-item
  prediction={}
  ii = KNNBasic(verbose=False,k=20,min_k=20, sim_options={'name': 'pearson', 'user_based': False, 'verbose' : True})#Veinatge de 20
  ii.fit(trainset)
  for novista in NOwatched_filmsId:
    title=movies_metadata[movies_metadata.movieId==novista]['title'].values[0]
    prediction[title]=ii.predict(uid=Id, iid=novista).est
  print('--------------------------------------------------------')
  print("Llistat de títols recomanats amb el métode item-item")
  print('--------------------------------------------------------')
  if N>len(prediction):#màxim titols recomanats és el total de pel·lícules predites
    N=len(prediction)
  for recomanacio in dict(sorted(prediction.items(), key = itemgetter(1), reverse = True)[:N]):
    print(recomanacio,', Predicció rating: ',dict(sorted(prediction.items(), key = itemgetter(1), reverse = True))[recomanacio])
  # SVD
  prediction={}
  svd = SVD(verbose=False, n_epochs=4)
  svd.fit(trainset)
  for novista in NOwatched_filmsId:
    title=movies_metadata[movies_metadata.movieId==novista]['title'].values[0]
    prediction[title]=svd.predict(uid=Id, iid=novista).est
  print('--------------------------------------------------------')
  print("Llistat de títols recomanats amb el métode SVD")
  print('--------------------------------------------------------')
  if N>len(prediction):#màxim titols recomanats és el total de pel·lícules predites
    N=len(prediction)
  for recomanacio in dict(sorted(prediction.items(), key = itemgetter(1), reverse = True)[:N]):
    print(recomanacio,', Predicció rating: ',dict(sorted(prediction.items(), key = itemgetter(1), reverse = True))[recomanacio])

Invoquem la funció amb l'**usuari 1 i 10 recomanacions**.

In [10]:
recommendations()

Escriu userId: 
1
Número de títols recomanats: 
10
Pel·lícules que més han agradat a l'usuari
--------------------------------------------------------
Seven (a.k.a. Se7en) (1995)
Usual Suspects, The (1995)
Bottle Rocket (1996)
Rob Roy (1995)
Canadian Bacon (1995)
Desperado (1995)
Billy Madison (1995)
Dumb & Dumber (Dumb and Dumber) (1994)
Star Wars: Episode IV - A New Hope (1977)
Tommy Boy (1995)
--------------------------------------------------------
Llistat de títols recomanats amb el métode user-user
--------------------------------------------------------
Shawshank Redemption, The (1994) , Predicció rating:  4.712915821853925
Departed, The (2006) , Predicció rating:  4.607072012329428
Casablanca (1942) , Predicció rating:  4.505273491125451
Godfather, The (1972) , Predicció rating:  4.452664816168339
Cool Hand Luke (1967) , Predicció rating:  4.4262852183803405
Spirited Away (Sen to Chihiro no kamikakushi) (2001) , Predicció rating:  4.407275961320861
Godfather: Part II, The (1974

El primer títol recomanat amb el mètode **user-user CF** *Shawshank Redemption, The (1994)* és el mateix que va ser recomanat en l'exercici 3 amb que vaig treballar amb només 4000 pel·lícules, bona senyal. Les pel·lícules recomanades amb els métodes **user-user** i **SVD** són molt conegudes i apareixen bastants recomanacions en comú utilitzant els 2 métodes. Les recomanacions amb el métode **item-item** són molt més desconegudes i hi ha més varietat.

#Comparació d'errors dels 3 mètodes.

Analitzem els errors **RMSE** i **MAE** dels 3 métodes utilitzant *cross-validation* amb un percentatge de dades de test del 20%:

In [ ]:
from surprise.model_selection import cross_validate

In [ ]:
cross_validate(uu, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0036  1.0036  1.0062  0.9930  1.0017  1.0016  0.0045  
MAE (testset)     0.7781  0.7793  0.7796  0.7721  0.7767  0.7772  0.0027  
Fit time          0.37    0.38    0.43    0.32    0.33    0.37    0.04    
Test time         1.62    4.25    1.62    1.58    1.83    2.18    1.04    


{'test_rmse': array([1.00356139, 1.00362605, 1.00617661, 0.9929949 , 1.00166874]),
 'test_mae': array([0.77807281, 0.77927112, 0.77962956, 0.77212664, 0.77665501]),
 'fit_time': (0.3693845272064209,
  0.3839902877807617,
  0.43327879905700684,
  0.3204009532928467,
  0.3282771110534668),
 'test_time': (1.619856357574463,
  4.249210357666016,
  1.6203114986419678,
  1.581559181213379,
  1.8273553848266602)}

In [ ]:
cross_validate(ii, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9958  0.9910  0.9891  0.9929  0.9882  0.9914  0.0027  
MAE (testset)     0.7793  0.7748  0.7733  0.7744  0.7728  0.7749  0.0023  
Fit time          10.92   10.10   10.69   9.95    10.93   10.52   0.41    
Test time         10.22   9.70    10.34   9.60    10.63   10.10   0.39    


{'test_rmse': array([0.99581387, 0.99100303, 0.98913506, 0.9928613 , 0.98818391]),
 'test_mae': array([0.7793037 , 0.77478525, 0.77325306, 0.77441224, 0.77283245]),
 'fit_time': (10.915890455245972,
  10.097280979156494,
  10.688978433609009,
  9.945682287216187,
  10.927297830581665),
 'test_time': (10.218065738677979,
  9.699293851852417,
  10.335005044937134,
  9.599459648132324,
  10.633747816085815)}

In [ ]:
cross_validate(svd, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9030  0.8877  0.8989  0.9003  0.9004  0.8981  0.0053  
MAE (testset)     0.6978  0.6873  0.6947  0.6973  0.6986  0.6951  0.0041  
Fit time          0.44    0.43    0.42    0.43    0.42    0.43    0.01    
Test time         0.16    0.16    0.15    0.23    0.36    0.21    0.08    


{'test_rmse': array([0.90302101, 0.88773748, 0.8989233 , 0.9003264 , 0.90039321]),
 'test_mae': array([0.69779214, 0.6873367 , 0.69468511, 0.69726053, 0.69856867]),
 'fit_time': (0.4415264129638672,
  0.4250316619873047,
  0.4216043949127197,
  0.43407559394836426,
  0.41987133026123047),
 'test_time': (0.1638047695159912,
  0.15626811981201172,
  0.15108299255371094,
  0.23026299476623535,
  0.36376500129699707)}

El métodes **user-user CF** i **item-item CF** presenten errors molt semblants, al voltant de 1 en RMSE i 0.78 MAE. **User-user CF** té un error RMSE unes mil·lèsimes per sobre i al voltant d'un 25% menys de desviació estàndar.

El mètode **SVC** té un error menor als altres, RMSE de 0.9 i MAE de 0.7. Amb aquest mètode dificilment errarem en més d'un punt la predicció.